In [2]:
import modern_robotics as mr
import numpy as np

np.set_printoptions(suppress=True)
from dataclasses import dataclass
from tpk4170.visualization import Ur5Visualizer

In [3]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )

In [5]:
class KR6:

    def __init__(self):
        self._M = np.array(
            [
                [-1.0, 0.0, 0.0, self.L1 + self.L2],
                [0.0, 0.0, 1.0, self.W1 + self.W2],
                [0.0, 1.0, 0.0, self.H1 - self.H2],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )

        self._SList = np.array(
            [
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, -self.H1, 0.0, 0.0],
                [0.0, 1.0, 0.0, -self.H1, 0.0, self.L1],
                [0.0, 1.0, 0.0, -self.H1, 0.0, self.L1 + self.L2],
                [0.0, 0.0, -1.0, -self.W1, self.L1 + self.L2, 0.0],
                [0.0, 1.0, 0.0, self.H2 - self.H1, 0.0, self.L1 + self.L2],
            ]
        ).T

    def fkin(self, theta):
        return mr.FKinSpace(self._M, self._SList, theta)


PI_HALF = np.pi / 2
PI = np.pi

In [6]:
kr6 = KR6()

AttributeError: 'KR6' object has no attribute 'L1'

In [24]:
theta = np.array([[0],
                   [0],
                   [PI_HALF],
                   [0],
                   [0],
                   [0]])

alpha = np.array([[0],
                  [-PI_HALF],
                  [0],
                  [PI_HALF],
                  [PI_HALF],
                  [-PI_HALF]])

d = np.array([[0],
              [400],
              [455],
              [0],
              [420],
              [80]])

a = np.array([[0],
              [25],
              [0],
              [35],
              [0],
              [0]])

dhList = np.concatenate((a, alpha, d, theta), axis=1)
dhList

array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [ 25.        ,  -1.57079633, 400.        ,   0.        ],
       [  0.        ,   0.        , 455.        ,   1.57079633],
       [ 35.        ,   1.57079633,   0.        ,   0.        ],
       [  0.        ,   1.57079633, 420.        ,   0.        ],
       [  0.        ,  -1.57079633,  80.        ,   0.        ]])

In [20]:
l1 = DHLink(*dhList[0])
l2 = DHLink(*dhList[1])
l3 = DHLink(*dhList[2])
l4 = DHLink(*dhList[3])
l5 = DHLink(*dhList[4])
l6 = DHLink(*dhList[5])

In [23]:
fk_dh_zero = (
    l1.matrix()
    @ l2.matrix()
    @ l3.matrix()
    @ l4.matrix()
    @ l5.matrix()
    @ l6.matrix()
)
fk_dh_zero

array([[  0.,  -0.,   1., 445.],
       [  0.,   1.,   0., 375.],
       [ -1.,   0.,   0., 365.],
       [  0.,   0.,   0.,   1.]])